In [10]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df = pd.read_csv('UCI_Credit_Card.csv')

In [16]:
df.columns

Index(['id', 'limit_bal', 'sex', 'education', 'marriage', 'age',
       'pay_stat_sep', 'pay_stat_aug', 'pay_stat_jul', 'pay_stat_jun',
       'pay_stat_may', 'pay_stat_apr', 'bill_amt1', 'bill_amt2', 'bill_amt3',
       'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1', 'pay_amt2',
       'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6',
       'default.payment.next.month'],
      dtype='object')

In [25]:
df.shape

(30000, 25)

In [12]:
# Combine other small categories  
df.loc[(df.EDUCATION == 5) | (df.EDUCATION == 6) | (df.EDUCATION == 0), 'EDUCATION'] = 4

In [13]:
# Combine to have 1:married, 2:single, 3:other
df.loc[df.MARRIAGE == 0, 'MARRIAGE'] = 3

In [14]:
df.rename({'PAY_0': 'pay_stat_sep', 'PAY_2': 'pay_stat_aug', 'PAY_3': 'pay_stat_jul', 'PAY_4': 'pay_stat_jun', 'PAY_5': 
           'pay_stat_may', 'PAY_6': 'pay_stat_apr'}, axis=1, inplace=True)

In [15]:
for col in df.columns:
    df.rename({'{}'.format(col): '{}'.format(col.lower())}, axis=1, inplace=True)

In [26]:
df['last_month_v_limit'] = df['bill_amt1']/df['limit_bal']

In [28]:
df['last_month_v_limit'].head() 

0    0.195650
1    0.022350
2    0.324878
3    0.939800
4    0.172340
Name: last_month_v_limit, dtype: float64

In [29]:
from scipy.stats.stats import pearsonr
pearsonr(df['default.payment.next.month'], df['last_month_v_limit'])

(0.08616810335391435, 1.5126308865122633e-50)

In [32]:
df['no_credit_used'] = np.where(df['pay_stat_sep']==-2, 1, 0)

In [34]:
df['full_payment_made'] = np.where(df['pay_stat_sep']==-1, 1, 0)

In [35]:
df['min_payment_made'] = np.where(df['pay_stat_sep']==0, 1, 0)